In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip3 install albumentations

In [3]:
from albumentations.core.transforms_interface import ImageOnlyTransform
from albumentations.pytorch import ToTensorV2
from albumentations import Compose,Resize

class Normalization(ImageOnlyTransform):
    """
    this class normalize the input image by dividing image values by 255

    """
    def __init__(self, always_apply: bool = True, p: float = 1):
        super().__init__(always_apply, p)
    
    def apply(self, img, **params) :
        return img/255.0
transform=Compose([
        Resize(height=224,width=224),
        Normalization(always_apply=True),
        ToTensorV2()])

In [4]:
from torch.utils.data import Dataset
from tqdm import tqdm
import cv2

def listImagesFilesInDir(dir_path:str)->list:
    files_list = [filename for filename in os.listdir(dir_path) if
                  (str.lower(filename[-4:]) in ['.jpg', '.png']) or (str.lower(filename[-5:]) in ['.jpeg','.tiff'])]
    return files_list

class CropSegDatasetInfer(Dataset):
    """
    dataloader that read dataset in the directory tree form only
    ---dataset
            |
            |----img

    """
    def __init__(self, dir_path,split, transforms=None):
        super(CropSegDatasetInfer, self).__init__()
        """
        dataset constructor

        attributes:-

        - dir_path : split path inside the dataset path
        - transforms : augmentation or transformation needs to be done on the image
        - data : list of paths that will be loaded during training item by item

        :param dir_path: dataset path
        :type dir_path: str
        :param split: split name
        :type split: str
        :param transform: transformations to be applied on each image
        :type transform: albumentations.Compose
        """
        self.dir_path = os.path.join(dir_path,split)
        self.transforms=transforms
        self.data = self.get_data_list()
    def get_data_list(self) :
        input_and_labels = []
        images_dir = os.path.join(self.dir_path)
        # labels_dir = os.path.join(self.dir_path, DictKeys.LBL.value)
        
        assert os.path.exists(images_dir), f'image directory @ {images_dir} does not exist'
        # assert os.path.exists(labels_dir), f'image directory @ {labels_dir} does not exist'
        images_names = listImagesFilesInDir(images_dir) #list of dataset images names
        
        print(f"[INFO] loading files names from {images_dir}")
        for file_name in tqdm(images_names):
            img_path = os.path.join(images_dir, file_name)
            # lbl_path = os.path.join(labels_dir, file_name)
            input_and_labels.append([file_name, img_path])

        return input_and_labels
    def __getitem__(self, index):
        """_summary_

        :param index: index of the item to be retrieved
        :type index: int
        :return: filename of the image read, image in float tensor
        :rtype: str,torch.FloatTensor
        """
        file_name, img_path = self.data[index]
        image = cv2.imread(img_path)#Image.open(img_path).convert('RGB')
        orig_size=image.shape
        if(self.transforms is not None):
            aug_out = self.transforms(image=image)
            image=aug_out['image']

        return file_name, image,orig_size
    def __len__(self):
        return len(self.data) 
dataset=CropSegDatasetInfer('/kaggle/input/hubmap-organ-segmentation/test_images','',transform)
print(dataset[0][2])

[INFO] loading files names from /kaggle/input/hubmap-organ-segmentation/test_images/


100%|██████████| 1/1 [00:00<00:00, 4084.04it/s]


(2023, 2023, 3)


In [6]:
import torch
from torch import nn
from torch.nn import functional as F
class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch, bn):
        super(double_conv, self).__init__()
        if bn:
            self.conv = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 3, padding=1),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_ch, out_ch, 3, padding=1),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True)
            )
        else:
            self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True)
        )


    def forward(self, x):
        x = self.conv(x)
        return x
class inconv(nn.Module):
    def __init__(self, in_ch, out_ch, bn):
        super(inconv, self).__init__()

        self.conv = double_conv(in_ch, out_ch, bn)

    def forward(self, x):
        x = self.conv(x)
        return x
class down(nn.Module):
    def __init__(self, in_ch, out_ch, bn):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            double_conv(in_ch, out_ch, bn)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x

class up(nn.Module):
    def __init__(self, in_ch, out_ch, bn, bilinear=True):
        super(up, self).__init__()

        #  would be a nice idea if the upsampling could be learned too,
        #  but my machine do not have enough memory to handle all those weights
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='nearest')
        else:
            self.up = nn.ConvTranspose2d(in_ch//2, in_ch//2, 2, stride=2)

        self.conv = double_conv(in_ch, out_ch, bn)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, (diffX // 2, diffX - diffX//2,
                        diffY // 2, diffY - diffY//2))
        
        # for padding issues, see 
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd

        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x
class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x
class UNetOutMaskClass(torch.nn.Module):
    """
    Vanilla unet model architecture
    """
    def __init__(self, n_classes,n_organs, batchnorm=False):
        super(UNetOutMaskClass, self).__init__()
        self.inc = inconv(3, 64, batchnorm)
        self.down1 = down(64, 128, batchnorm)
        self.down2 = down(128, 256, batchnorm)
        self.down3 = down(256, 512, batchnorm)
        self.down4 = down(512, 512, batchnorm)
        self.up1 = up(1024, 256, batchnorm)
        self.up2 = up(512, 128, batchnorm)
        self.up3 = up(256, 64, batchnorm)
        self.up4 = up(128, 64, batchnorm)
        self.outc = outconv(64, n_classes)
        self.pool=torch.nn.AdaptiveAvgPool2d(1)
        self.fc1=torch.nn.Linear(64,64)
        self.fc2=torch.nn.Linear(64,64)
        self.outorgan=torch.nn.Linear(64,n_organs)

    def forward(self,x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        pooled=self.pool(x)
        pooled=torch.flatten(pooled,start_dim=1)
        # print(x.shape)
        x = self.outc(x)
        # print(pooled.shape)
        pooled=self.fc2(self.fc1(pooled))
        
        organ=self.outorgan(pooled)
        return x,organ
class UNet(torch.nn.Module):
    """
    Vanilla unet model architecture
    """
    def __init__(self, n_classes, batchnorm=False):
        super(UNet, self).__init__()
        self.inc = inconv(3, 64, batchnorm)
        self.down1 = down(64, 128, batchnorm)
        self.down2 = down(128, 256, batchnorm)
        self.down3 = down(256, 512, batchnorm)
        self.down4 = down(512, 512, batchnorm)
        self.up1 = up(1024, 256, batchnorm)
        self.up2 = up(512, 128, batchnorm)
        self.up3 = up(256, 64, batchnorm)
        self.up4 = up(128, 64, batchnorm)
        self.outc = outconv(64, n_classes)
#         self.pool=torch.nn.AdaptiveAvgPool2d(1)
#         self.fc1=torch.nn.Linear(64,64)
#         self.fc2=torch.nn.Linear(64,64)
#         self.outorgan=torch.nn.Linear(64,n_organs)

    def forward(self,x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
#         pooled=self.pool(x)
#         pooled=torch.flatten(pooled,start_dim=1)
        # print(x.shape)
        x = self.outc(x)
        # print(pooled.shape)
#         pooled=self.fc2(self.fc1(pooled))
        
#         organ=self.outorgan(pooled)
        return x
class UNetLight(torch.nn.Module):
    """
    Vanilla unet model architecture but with less parameters
    """
    def __init__(self, n_classes, batchnorm=False):
        super(UNetLight, self).__init__()
        self.inc = inconv(3, 8, batchnorm)
        self.down1 = down(8, 16, batchnorm)
        self.down2 = down(16, 32, batchnorm)
        self.down3 = down(32, 64, batchnorm)
        self.down4 = down(64, 64, batchnorm)
        self.up1 = up(128, 32, batchnorm)
        self.up2 = up(64, 16, batchnorm)
        self.up3 = up(32, 8, batchnorm)
        self.up4 = up(16, 8, batchnorm)
        self.outc = outconv(8, n_classes)

    def forward(self,x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x
model=UNetOutMaskClass(2,5,True)#.to('cuda')
model.load_state_dict(torch.load('/kaggle/input/models-hac/HAC-81-UNET_FULL_WITH_CLASS-best_weights.ckpt',map_location='cpu')['model_state_dict'])

<All keys matched successfully>

In [7]:
model

UNetOutMaskClass(
  (inc): inconv(
    (conv): double_conv(
      (conv): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
  )
  (down1): down(
    (mpconv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): double_conv(
        (conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4)

In [8]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
 
import numpy as np
model.eval()
def vis_mask(mask):
    import matplotlib.pyplot as plt
    plt.figure(figsize=(15, 15))
    plt.imshow(mask)
    plt.axis("off")
    plt.show()
    return 
id_rle={'id':[],'rle':[]}
with torch.no_grad():
    for ii,(file_name,image,original_size) in tqdm(enumerate(dataset),total=len(dataset)):
        image=image.unsqueeze(dim=0).float()#.to('cuda')
        mask,_=model(image)
        mask=torch.argmax(mask,dim=1)
        mask=np.uint8(mask.detach().squeeze(dim=0).numpy())*255
        print(mask.shape)
        mask=cv2.resize(mask,dsize=original_size[:-1],interpolation=cv2.INTER_NEAREST)
#         vis_mask(mask)
        id_rle['id'].append(file_name.split('.')[0])
        id_rle['rle'].append(mask2rle(mask))
pd.DataFrame(id_rle).to_csv('submission.csv',index=False)

100%|██████████| 1/1 [00:00<00:00,  1.59it/s]

(224, 224)
